In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2
from PIL import Image
from tqdm.notebook import tqdm
from scipy.ndimage import label
import cupy as cp

In [ ]:
def normalize_x(image):
    return image / 127.5 - 1

def denormalize_x(image):
    return (image + 1) * 127.5

def normalize_y(image):
    return image / 255

def denormalize_y(image):
    return image * 255

In [ ]:
def load_X_gray(folder_path):
    
    image_files = []

    #image_files = os.listdir(folder_path)
       
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else :
            pass
        
    image_files.sort()
    
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)
    
    #image_files = image_files[1:]
    images = np.zeros((len(image_files), img.shape[0], img.shape[1], 1), np.float32)
    for i, image_file in tqdm(enumerate(image_files)):
        image = cv2.imread(folder_path + os.sep + image_file, cv2.IMREAD_GRAYSCALE)
        #image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
        image = image[:, :, np.newaxis]
        images[i] = normalize_x(image)
    
    print(images.shape)
    
    return images, image_files


def load_Y_gray(folder_path, thresh = None , normalize = True, g_size = None):
    image_files = []
    #image_files = os.listdir(folder_path)
    
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else:
            pass
        
    image_files.sort()
    
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)
    
    images = np.zeros(
        (len(image_files), img.shape[0], img.shape[1], 1) ,np.float32
    )
    
    for i , image_file in tqdm(enumerate(image_files)):
        image = cv2.imread(
            folder_path + os.sep + image_file ,
            cv2.IMREAD_GRAYSCALE
        )
        #print(image.shape)
        
        # ぼかし処理
        if g_size:
            image = cv2.GaussianBlur(
                image, (g_size, g_size), 0
            )        
        
        if thresh:
            ret , image = cv2.threshold(image , thresh , 255 , cv2.THRESH_BINARY)
        image = image[ : , : , np.newaxis]
        if normalize:
            images[i] = normalize_y(image)
        else:
            images[i] = image
            
    print(images.shape)
    
    return images , image_files

# shCtrl

## cropped_001

In [ ]:
lamellar_imgs, _ = load_Y_gray("Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_final//lamellar/")
tubular_imgs, _ = load_Y_gray("Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_final//tubular/")


In [ ]:
merged_imgs = np.zeros(
    (lamellar_imgs.shape[0], lamellar_imgs.shape[1], lamellar_imgs.shape[2], 3), 
    np.float32
)
merged_imgs[:,:,:,1] = lamellar_imgs[:,:,:,0]*255
merged_imgs[:,:,:,0] = tubular_imgs[:,:,:,0]*255

In [ ]:
label_imgs, label_num = label(lamellar_imgs)
boxels_array = [ np.count_nonzero(label_imgs == i) for i in range(label_num) ]

In [ ]:
del lamellar_imgs, tubular_imgs

In [ ]:
for i in range(0, label_num+1):
    print(f"{str(i).zfill(4)} / {label_num}")
    if np.count_nonzero(label_imgs == i) <= 200:
        merged_imgs[:,:,:,1]  = np.where(
            label_imgs[:,:,:,0] == i,
            0,
            merged_imgs[:,:,:,1]
        )
        
        merged_imgs[:,:,:,0]  = np.where(
            label_imgs[:,:,:,0] == i,
            1,
            merged_imgs[:,:,:,0]
        )    

In [ ]:
merged_imgs[:,:,:,1] = np.where(
    merged_imgs[:,:,:,0] > 0,
    0,
    merged_imgs[:,:,:,1]
)

In [ ]:
label_imgs, label_num = label(merged_imgs[:,:,:,0])
boxels_array = [ np.count_nonzero(label_imgs == i) for i in range(label_num) ]

In [ ]:
for i in range(0, label_num+1):
    print(f"{str(i).zfill(4)} / {label_num}")
    if np.count_nonzero(label_imgs == i) <= 20:
        merged_imgs[:,:,:,0]  = np.where(
            label_imgs == i,
            0,
            merged_imgs[:,:,:,0]
        )

In [ ]:
out_path = "Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_final/"

os.makedirs(f"{out_path}/lamellar_with_postprocessing", exist_ok = True)
os.makedirs(f"{out_path}/tubular_with_postprocessing", exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_path}/lamellar_with_postprocessing/{str(i).zfill(4)}.png",
        merged_imgs[i][:,:,1]
    )
    cv2.imwrite(
        f"{out_path}/tubular_with_postprocessing/{str(i).zfill(4)}.png",
        merged_imgs[i][:,:,0]
    )    

## cropped_002

In [ ]:
lamellar_imgs, _ = load_Y_gray("Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_final//lamellar/")
tubular_imgs, _ = load_Y_gray("Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_final//tubular/")


In [ ]:
merged_imgs = np.zeros(
    (lamellar_imgs.shape[0], lamellar_imgs.shape[1], lamellar_imgs.shape[2], 3), 
    np.float32
)
merged_imgs[:,:,:,1] = lamellar_imgs[:,:,:,0]*255
merged_imgs[:,:,:,0] = tubular_imgs[:,:,:,0]*255

In [ ]:
label_imgs, label_num = label(lamellar_imgs)
boxels_array = [ np.count_nonzero(label_imgs == i) for i in range(label_num) ]

In [ ]:
del lamellar_imgs, tubular_imgs

In [ ]:
for i in range(0, label_num+1):
    print(f"{str(i).zfill(4)} / {label_num}")
    if np.count_nonzero(label_imgs == i) <= 200:
        merged_imgs[:,:,:,1]  = np.where(
            label_imgs[:,:,:,0] == i,
            0,
            merged_imgs[:,:,:,1]
        )
        
        merged_imgs[:,:,:,0]  = np.where(
            label_imgs[:,:,:,0] == i,
            1,
            merged_imgs[:,:,:,0]
        )    

In [ ]:
merged_imgs[:,:,:,1] = np.where(
    merged_imgs[:,:,:,0] > 0,
    0,
    merged_imgs[:,:,:,1]
)

In [ ]:
label_imgs, label_num = label(merged_imgs[:,:,:,0])
boxels_array = [ np.count_nonzero(label_imgs == i) for i in range(label_num) ]

In [ ]:
for i in range(0, label_num+1):
    print(f"{str(i).zfill(4)} / {label_num}")
    if np.count_nonzero(label_imgs == i) <= 20:
        merged_imgs[:,:,:,0]  = np.where(
            label_imgs == i,
            0,
            merged_imgs[:,:,:,0]
        )

In [ ]:
out_path = "Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_final/"

os.makedirs(f"{out_path}/lamellar_with_postprocessing", exist_ok = True)
os.makedirs(f"{out_path}/tubular_with_postprocessing", exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_path}/lamellar_with_postprocessing/{str(i).zfill(4)}.png",
        merged_imgs[i][:,:,1]
    )
    cv2.imwrite(
        f"{out_path}/tubular_with_postprocessing/{str(i).zfill(4)}.png",
        merged_imgs[i][:,:,0]
    )    

## cropped_003

In [ ]:
lamellar_imgs, _ = load_Y_gray("Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_final//lamellar/")
tubular_imgs, _ = load_Y_gray("Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_final//tubular/")


In [ ]:
merged_imgs = np.zeros(
    (lamellar_imgs.shape[0], lamellar_imgs.shape[1], lamellar_imgs.shape[2], 3), 
    np.float32
)
merged_imgs[:,:,:,1] = lamellar_imgs[:,:,:,0]*255
merged_imgs[:,:,:,0] = tubular_imgs[:,:,:,0]*255

In [ ]:
label_imgs, label_num = label(lamellar_imgs)
boxels_array = [ np.count_nonzero(label_imgs == i) for i in range(label_num) ]

In [ ]:
del lamellar_imgs, tubular_imgs

In [ ]:
for i in range(0, label_num+1):
    print(f"{str(i).zfill(4)} / {label_num}")
    if np.count_nonzero(label_imgs == i) <= 200:
        merged_imgs[:,:,:,1]  = np.where(
            label_imgs[:,:,:,0] == i,
            0,
            merged_imgs[:,:,:,1]
        )
        
        merged_imgs[:,:,:,0]  = np.where(
            label_imgs[:,:,:,0] == i,
            1,
            merged_imgs[:,:,:,0]
        )    

In [ ]:
merged_imgs[:,:,:,1] = np.where(
    merged_imgs[:,:,:,0] > 0,
    0,
    merged_imgs[:,:,:,1]
)

In [ ]:
label_imgs, label_num = label(merged_imgs[:,:,:,0])
boxels_array = [ np.count_nonzero(label_imgs == i) for i in range(label_num) ]

In [ ]:
for i in range(0, label_num+1):
    print(f"{str(i).zfill(4)} / {label_num}")
    if np.count_nonzero(label_imgs == i) <= 20:
        merged_imgs[:,:,:,0]  = np.where(
            label_imgs == i,
            0,
            merged_imgs[:,:,:,0]
        )

In [ ]:
out_path = "Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_final/"

os.makedirs(f"{out_path}/lamellar_with_postprocessing", exist_ok = True)
os.makedirs(f"{out_path}/tubular_with_postprocessing", exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_path}/lamellar_with_postprocessing/{str(i).zfill(4)}.png",
        merged_imgs[i][:,:,1]
    )
    cv2.imwrite(
        f"{out_path}/tubular_with_postprocessing/{str(i).zfill(4)}.png",
        merged_imgs[i][:,:,0]
    )    

## cropped_004

In [ ]:
lamellar_imgs, _ = load_Y_gray("Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_final//lamellar/")
tubular_imgs, _ = load_Y_gray("Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_final//tubular/")


In [ ]:
merged_imgs = np.zeros(
    (lamellar_imgs.shape[0], lamellar_imgs.shape[1], lamellar_imgs.shape[2], 3), 
    np.float32
)
merged_imgs[:,:,:,1] = lamellar_imgs[:,:,:,0]*255
merged_imgs[:,:,:,0] = tubular_imgs[:,:,:,0]*255

In [ ]:
label_imgs, label_num = label(lamellar_imgs)
boxels_array = [ np.count_nonzero(label_imgs == i) for i in range(label_num) ]

In [ ]:
del lamellar_imgs, tubular_imgs

In [ ]:
for i in range(0, label_num+1):
    print(f"{str(i).zfill(4)} / {label_num}")
    if np.count_nonzero(label_imgs == i) <= 200:
        merged_imgs[:,:,:,1]  = np.where(
            label_imgs[:,:,:,0] == i,
            0,
            merged_imgs[:,:,:,1]
        )
        
        merged_imgs[:,:,:,0]  = np.where(
            label_imgs[:,:,:,0] == i,
            1,
            merged_imgs[:,:,:,0]
        )    

In [ ]:
merged_imgs[:,:,:,1] = np.where(
    merged_imgs[:,:,:,0] > 0,
    0,
    merged_imgs[:,:,:,1]
)

In [ ]:
label_imgs, label_num = label(merged_imgs[:,:,:,0])
boxels_array = [ np.count_nonzero(label_imgs == i) for i in range(label_num) ]

In [ ]:
for i in range(0, label_num+1):
    print(f"{str(i).zfill(4)} / {label_num}")
    if np.count_nonzero(label_imgs == i) <= 20:
        merged_imgs[:,:,:,0]  = np.where(
            label_imgs == i,
            0,
            merged_imgs[:,:,:,0]
        )

In [ ]:
out_path = "Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_final/"

os.makedirs(f"{out_path}/lamellar_with_postprocessing", exist_ok = True)
os.makedirs(f"{out_path}/tubular_with_postprocessing", exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_path}/lamellar_with_postprocessing/{str(i).zfill(4)}.png",
        merged_imgs[i][:,:,1]
    )
    cv2.imwrite(
        f"{out_path}/tubular_with_postprocessing/{str(i).zfill(4)}.png",
        merged_imgs[i][:,:,0]
    )    

## cropped_005

In [ ]:
lamellar_imgs, _ = load_Y_gray("Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_final//lamellar/")
tubular_imgs, _ = load_Y_gray("Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_final//tubular/")


In [ ]:
merged_imgs = np.zeros(
    (lamellar_imgs.shape[0], lamellar_imgs.shape[1], lamellar_imgs.shape[2], 3), 
    np.float32
)
merged_imgs[:,:,:,1] = lamellar_imgs[:,:,:,0]*255
merged_imgs[:,:,:,0] = tubular_imgs[:,:,:,0]*255

In [ ]:
label_imgs, label_num = label(lamellar_imgs)
boxels_array = [ np.count_nonzero(label_imgs == i) for i in range(label_num) ]

In [ ]:
del lamellar_imgs, tubular_imgs

In [ ]:
for i in range(0, label_num+1):
    print(f"{str(i).zfill(4)} / {label_num}")
    if np.count_nonzero(label_imgs == i) <= 200:
        merged_imgs[:,:,:,1]  = np.where(
            label_imgs[:,:,:,0] == i,
            0,
            merged_imgs[:,:,:,1]
        )
        
        merged_imgs[:,:,:,0]  = np.where(
            label_imgs[:,:,:,0] == i,
            1,
            merged_imgs[:,:,:,0]
        )    

In [ ]:
merged_imgs[:,:,:,1] = np.where(
    merged_imgs[:,:,:,0] > 0,
    0,
    merged_imgs[:,:,:,1]
)

In [ ]:
label_imgs, label_num = label(merged_imgs[:,:,:,0])
boxels_array = [ np.count_nonzero(label_imgs == i) for i in range(label_num) ]

In [ ]:
for i in range(0, label_num+1):
    print(f"{str(i).zfill(4)} / {label_num}")
    if np.count_nonzero(label_imgs == i) <= 20:
        merged_imgs[:,:,:,0]  = np.where(
            label_imgs == i,
            0,
            merged_imgs[:,:,:,0]
        )

In [ ]:
out_path = "Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_final/"

os.makedirs(f"{out_path}/lamellar_with_postprocessing", exist_ok = True)
os.makedirs(f"{out_path}/tubular_with_postprocessing", exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_path}/lamellar_with_postprocessing/{str(i).zfill(4)}.png",
        merged_imgs[i][:,:,1]
    )
    cv2.imwrite(
        f"{out_path}/tubular_with_postprocessing/{str(i).zfill(4)}.png",
        merged_imgs[i][:,:,0]
    )    

# shOPA1

## cropped_001

In [ ]:
lamellar_imgs, _ = load_Y_gray("Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_final//lamellar/")
tubular_imgs, _ = load_Y_gray("Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_final//tubular/")


In [ ]:
merged_imgs = np.zeros(
    (lamellar_imgs.shape[0], lamellar_imgs.shape[1], lamellar_imgs.shape[2], 3), 
    np.float32
)
merged_imgs[:,:,:,1] = lamellar_imgs[:,:,:,0]*255
merged_imgs[:,:,:,0] = tubular_imgs[:,:,:,0]*255

In [ ]:
label_imgs, label_num = label(lamellar_imgs)
boxels_array = [ np.count_nonzero(label_imgs == i) for i in range(label_num) ]

In [ ]:
del lamellar_imgs, tubular_imgs

In [ ]:
for i in range(0, label_num+1):
    print(f"{str(i).zfill(4)} / {label_num}")
    if np.count_nonzero(label_imgs == i) <= 200:
        merged_imgs[:,:,:,1]  = np.where(
            label_imgs[:,:,:,0] == i,
            0,
            merged_imgs[:,:,:,1]
        )
        
        merged_imgs[:,:,:,0]  = np.where(
            label_imgs[:,:,:,0] == i,
            1,
            merged_imgs[:,:,:,0]
        )    

In [ ]:
merged_imgs[:,:,:,1] = np.where(
    merged_imgs[:,:,:,0] > 0,
    0,
    merged_imgs[:,:,:,1]
)

In [ ]:
label_imgs, label_num = label(merged_imgs[:,:,:,0])
boxels_array = [ np.count_nonzero(label_imgs == i) for i in range(label_num) ]

In [ ]:
for i in range(0, label_num+1):
    print(f"{str(i).zfill(4)} / {label_num}")
    if np.count_nonzero(label_imgs == i) <= 20:
        merged_imgs[:,:,:,0]  = np.where(
            label_imgs == i,
            0,
            merged_imgs[:,:,:,0]
        )

In [ ]:
out_path = "Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_final/"

os.makedirs(f"{out_path}/lamellar_with_postprocessing", exist_ok = True)
os.makedirs(f"{out_path}/tubular_with_postprocessing", exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_path}/lamellar_with_postprocessing/{str(i).zfill(4)}.png",
        merged_imgs[i][:,:,1]
    )
    cv2.imwrite(
        f"{out_path}/tubular_with_postprocessing/{str(i).zfill(4)}.png",
        merged_imgs[i][:,:,0]
    )    

## cropped_002

In [ ]:
lamellar_imgs, _ = load_Y_gray("Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_final//lamellar/")
tubular_imgs, _ = load_Y_gray("Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_final//tubular/")


In [ ]:
merged_imgs = np.zeros(
    (lamellar_imgs.shape[0], lamellar_imgs.shape[1], lamellar_imgs.shape[2], 3), 
    np.float32
)
merged_imgs[:,:,:,1] = lamellar_imgs[:,:,:,0]*255
merged_imgs[:,:,:,0] = tubular_imgs[:,:,:,0]*255

In [ ]:
label_imgs, label_num = label(lamellar_imgs)
boxels_array = [ np.count_nonzero(label_imgs == i) for i in range(label_num) ]

In [ ]:
del lamellar_imgs, tubular_imgs

In [ ]:
for i in range(0, label_num+1):
    print(f"{str(i).zfill(4)} / {label_num}")
    if np.count_nonzero(label_imgs == i) <= 200:
        merged_imgs[:,:,:,1]  = np.where(
            label_imgs[:,:,:,0] == i,
            0,
            merged_imgs[:,:,:,1]
        )
        
        merged_imgs[:,:,:,0]  = np.where(
            label_imgs[:,:,:,0] == i,
            1,
            merged_imgs[:,:,:,0]
        )    

In [ ]:
merged_imgs[:,:,:,1] = np.where(
    merged_imgs[:,:,:,0] > 0,
    0,
    merged_imgs[:,:,:,1]
)

In [ ]:
label_imgs, label_num = label(merged_imgs[:,:,:,0])
boxels_array = [ np.count_nonzero(label_imgs == i) for i in range(label_num) ]

In [ ]:
for i in range(0, label_num+1):
    print(f"{str(i).zfill(4)} / {label_num}")
    if np.count_nonzero(label_imgs == i) <= 20:
        merged_imgs[:,:,:,0]  = np.where(
            label_imgs == i,
            0,
            merged_imgs[:,:,:,0]
        )

In [ ]:
out_path = "Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_final/"

os.makedirs(f"{out_path}/lamellar_with_postprocessing", exist_ok = True)
os.makedirs(f"{out_path}/tubular_with_postprocessing", exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_path}/lamellar_with_postprocessing/{str(i).zfill(4)}.png",
        merged_imgs[i][:,:,1]
    )
    cv2.imwrite(
        f"{out_path}/tubular_with_postprocessing/{str(i).zfill(4)}.png",
        merged_imgs[i][:,:,0]
    )    

## cropped_003

In [ ]:
lamellar_imgs, _ = load_Y_gray("Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/merged_cristae_final//lamellar/")
tubular_imgs, _ = load_Y_gray("Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/merged_cristae_final//tubular/")


In [ ]:
merged_imgs = np.zeros(
    (lamellar_imgs.shape[0], lamellar_imgs.shape[1], lamellar_imgs.shape[2], 3), 
    np.float32
)
merged_imgs[:,:,:,1] = lamellar_imgs[:,:,:,0]*255
merged_imgs[:,:,:,0] = tubular_imgs[:,:,:,0]*255

In [ ]:
label_imgs, label_num = label(lamellar_imgs)
boxels_array = [ np.count_nonzero(label_imgs == i) for i in range(label_num) ]

In [ ]:
del lamellar_imgs, tubular_imgs

In [ ]:
for i in range(0, label_num+1):
    print(f"{str(i).zfill(4)} / {label_num}")
    if np.count_nonzero(label_imgs == i) <= 200:
        merged_imgs[:,:,:,1]  = np.where(
            label_imgs[:,:,:,0] == i,
            0,
            merged_imgs[:,:,:,1]
        )
        
        merged_imgs[:,:,:,0]  = np.where(
            label_imgs[:,:,:,0] == i,
            1,
            merged_imgs[:,:,:,0]
        )    

In [ ]:
merged_imgs[:,:,:,1] = np.where(
    merged_imgs[:,:,:,0] > 0,
    0,
    merged_imgs[:,:,:,1]
)

In [ ]:
label_imgs, label_num = label(merged_imgs[:,:,:,0])
boxels_array = [ np.count_nonzero(label_imgs == i) for i in range(label_num) ]

In [ ]:
for i in range(0, label_num+1):
    print(f"{str(i).zfill(4)} / {label_num}")
    if np.count_nonzero(label_imgs == i) <= 20:
        merged_imgs[:,:,:,0]  = np.where(
            label_imgs == i,
            0,
            merged_imgs[:,:,:,0]
        )

In [ ]:
out_path = "Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/merged_cristae_final/"

os.makedirs(f"{out_path}/lamellar_with_postprocessing", exist_ok = True)
os.makedirs(f"{out_path}/tubular_with_postprocessing", exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_path}/lamellar_with_postprocessing/{str(i).zfill(4)}.png",
        merged_imgs[i][:,:,1]
    )
    cv2.imwrite(
        f"{out_path}/tubular_with_postprocessing/{str(i).zfill(4)}.png",
        merged_imgs[i][:,:,0]
    )    

## cropped_004

In [ ]:
lamellar_imgs, _ = load_Y_gray("Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/merged_cristae_final//lamellar/")
tubular_imgs, _ = load_Y_gray("Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/merged_cristae_final//tubular/")


In [ ]:
merged_imgs = np.zeros(
    (lamellar_imgs.shape[0], lamellar_imgs.shape[1], lamellar_imgs.shape[2], 3), 
    np.float32
)
merged_imgs[:,:,:,1] = lamellar_imgs[:,:,:,0]*255
merged_imgs[:,:,:,0] = tubular_imgs[:,:,:,0]*255

In [ ]:
label_imgs, label_num = label(lamellar_imgs)
boxels_array = [ np.count_nonzero(label_imgs == i) for i in range(label_num) ]

In [ ]:
del lamellar_imgs, tubular_imgs

In [ ]:
for i in range(0, label_num+1):
    print(f"{str(i).zfill(4)} / {label_num}")
    if np.count_nonzero(label_imgs == i) <= 200:
        merged_imgs[:,:,:,1]  = np.where(
            label_imgs[:,:,:,0] == i,
            0,
            merged_imgs[:,:,:,1]
        )
        
        merged_imgs[:,:,:,0]  = np.where(
            label_imgs[:,:,:,0] == i,
            1,
            merged_imgs[:,:,:,0]
        )    

In [ ]:
merged_imgs[:,:,:,1] = np.where(
    merged_imgs[:,:,:,0] > 0,
    0,
    merged_imgs[:,:,:,1]
)

In [ ]:
label_imgs, label_num = label(merged_imgs[:,:,:,0])
boxels_array = [ np.count_nonzero(label_imgs == i) for i in range(label_num) ]

In [ ]:
for i in range(0, label_num+1):
    print(f"{str(i).zfill(4)} / {label_num}")
    if np.count_nonzero(label_imgs == i) <= 20:
        merged_imgs[:,:,:,0]  = np.where(
            label_imgs == i,
            0,
            merged_imgs[:,:,:,0]
        )

In [ ]:
out_path = "Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/merged_cristae_final/"

os.makedirs(f"{out_path}/lamellar_with_postprocessing", exist_ok = True)
os.makedirs(f"{out_path}/tubular_with_postprocessing", exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_path}/lamellar_with_postprocessing/{str(i).zfill(4)}.png",
        merged_imgs[i][:,:,1]
    )
    cv2.imwrite(
        f"{out_path}/tubular_with_postprocessing/{str(i).zfill(4)}.png",
        merged_imgs[i][:,:,0]
    )    

## cropped_005

In [ ]:
lamellar_imgs, _ = load_Y_gray("Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_final//lamellar/")
tubular_imgs, _ = load_Y_gray("Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_final//tubular/")


In [ ]:
merged_imgs = np.zeros(
    (lamellar_imgs.shape[0], lamellar_imgs.shape[1], lamellar_imgs.shape[2], 3), 
    np.float32
)
merged_imgs[:,:,:,1] = lamellar_imgs[:,:,:,0]*255
merged_imgs[:,:,:,0] = tubular_imgs[:,:,:,0]*255

In [ ]:
label_imgs, label_num = label(lamellar_imgs)
boxels_array = [ np.count_nonzero(label_imgs == i) for i in range(label_num) ]

In [ ]:
del lamellar_imgs, tubular_imgs

In [ ]:
for i in range(0, label_num+1):
    print(f"{str(i).zfill(4)} / {label_num}")
    if np.count_nonzero(label_imgs == i) <= 200:
        merged_imgs[:,:,:,1]  = np.where(
            label_imgs[:,:,:,0] == i,
            0,
            merged_imgs[:,:,:,1]
        )
        
        merged_imgs[:,:,:,0]  = np.where(
            label_imgs[:,:,:,0] == i,
            1,
            merged_imgs[:,:,:,0]
        )    

In [ ]:
merged_imgs[:,:,:,1] = np.where(
    merged_imgs[:,:,:,0] > 0,
    0,
    merged_imgs[:,:,:,1]
)

In [ ]:
label_imgs, label_num = label(merged_imgs[:,:,:,0])
boxels_array = [ np.count_nonzero(label_imgs == i) for i in range(label_num) ]

In [ ]:
for i in range(0, label_num+1):
    print(f"{str(i).zfill(4)} / {label_num}")
    if np.count_nonzero(label_imgs == i) <= 20:
        merged_imgs[:,:,:,0]  = np.where(
            label_imgs == i,
            0,
            merged_imgs[:,:,:,0]
        )

In [ ]:
out_path = "Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_final/"

os.makedirs(f"{out_path}/lamellar_with_postprocessing", exist_ok = True)
os.makedirs(f"{out_path}/tubular_with_postprocessing", exist_ok = True)

for i in range(merged_imgs.shape[0]):
    cv2.imwrite(
        f"{out_path}/lamellar_with_postprocessing/{str(i).zfill(4)}.png",
        merged_imgs[i][:,:,1]
    )
    cv2.imwrite(
        f"{out_path}/tubular_with_postprocessing/{str(i).zfill(4)}.png",
        merged_imgs[i][:,:,0]
    )    